This script is for calculating the specific surface area ($S_{BET}$) of a solid sample, including error from Inverse Gas Chromatography measurements, specifically those done on the SEA (Surface energy analyser). Measured parameters are imported from a csv file and the specific surface area of the sample is printed at the end of the document along with visualisations of the adsorption isotherm and BET plot.

In [63]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
from scipy import stats, constants
import matplotlib.pyplot as plt
%matplotlib inline

# Organising data

In [64]:
# Reads csv input file and skips rows to where data start
input_file = raw_input('Copy path to input file here \nmake sure to insert extra "\\" and remove "" \n')
df = pd.read_csv(input_file, skiprows = 17)

Copy path to input file here 
make sure to insert extra "\" and remove "" 
C:\\Users\\tcd\\Documents\\PhD_scripts\\Surface area calculation\\surface_area_calculation_example_data.csv


In [65]:
# Removing of solvents that arent used in calculation
df= df[(df.Solvent != 'n/a') & (df.Solvent != 'Methane')]
df

,ID,Injection Name,Solvent,Injection Time [ms],Duration [min],Target Fractional Surface Coverage,Actual Fractional Surface Coverage,Column Temperature [Kelvin],Column Pressure Drop [Torr],Exit Flow Rate [sccm],...,Net Ret Volume (Max) [ml/g],Net Ret Time (Com) [min],Net Ret Volume (Com) [ml/g],Partial Pressure [Torr],Pres Ret Volume (Max) [mMol/(g�Torr)],Pres Ret Volume (Com) [mMol/(g�Torr)],Amount [mMol/g],Amount [mMol],Amount (Max) [mMol/g],Amount (Com) [mMol/g]
3,4,injection1,OCTANE,2908,60,0.05,0.000000,293.030255,113.264702,20.007500,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,5,injection2,OCTANE,5326,50,0.1,0.000000,293.138455,113.193153,19.992500,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,6,injection3,OCTANE,7376,45,0.15,0.000000,293.147285,113.293301,19.995001,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,7,injection4,OCTANE,9197,40,0.2,0.131797,293.147840,113.264702,20.002500,...,10546.601140,8.844827,25203.010090,0.019239,0.576888,1.378579,0.028781,0.000201,0.011099,0.026523
7,8,injection5,OCTANE,10929,35,0.25,0.167467,293.149920,113.121601,19.977500,...,7262.437478,6.562913,18679.246390,0.036481,0.397245,1.021729,0.036570,0.000256,0.017948,0.044139
8,9,injection6,OCTANE,12705,30,0.3,0.207708,293.145860,113.293301,19.990000,...,5320.880754,5.055532,14396.190190,0.064325,0.291049,0.787462,0.045358,0.000318,0.026052,0.066065
9,10,injection7,OCTANE,14641,25,0.35,0.231363,293.152291,113.193150,20.004999,...,4247.511731,3.509818,10002.955540,0.098494,0.232331,0.547143,0.050524,0.000354,0.033990,0.084760
10,11,injection8,OCTANE,16825,20,0.4,0.282646,293.147820,113.264652,19.980000,...,3502.106118,3.058096,8704.131039,0.151858,0.191562,0.476107,0.061723,0.000432,0.044213,0.110168
11,12,injection9,OCTANE,19318,17,0.45,0.328517,293.154195,113.350548,19.990001,...,3086.241110,2.613740,7442.855735,0.213370,0.168811,0.407108,0.071740,0.000502,0.054597,0.135210
12,13,injection10,OCTANE,22148,15,0.5,0.358625,293.155475,114.766899,19.987500,...,2831.846623,2.279481,6484.375513,0.265711,0.154895,0.354680,0.078314,0.000548,0.062704,0.153774


In [66]:
# Renaming columns with funny character
df.rename(columns ={'Peak Area [\xb5V\x95min]': 'peak_area', 'Peak Max (Signal) [\xb5V]': 'peak_max_signal'},inplace=True)

# Removing unwanted rows
df = df[df.peak_area != 0]

#  James-Martin pressure correction, j

$$j=\frac{3}{2}\frac{(\frac{p_{in}}{p_{out}})^{2}-1}{(\frac{p_{in}}{p_{out}})^{3}-1}$$

In [67]:
# Atmospheric pressure in torr
p_out = 760

# input pressure is pressure drop + atomospheric pressure
p_in = df['Column Pressure Drop [Torr]'] + p_out

j = (1.5*(((p_in/p_out)**2 -1)/((p_in/p_out)**3 -1)))

# Net Retention Volume, Vn ($\frac{mL}{g}$)

$$V_{N}=\frac{j}{m}F(t_{r}-t_{0})(\frac{T}{273.15})$$

In [80]:
# Creating input for sample mass and change its units
m_df = pd.read_csv(input_file,nrows=1,header=None,skiprows=4)
m_df.head()
m=m_df.ix[:,1:1]
m

#m = m / 1000 # mg to g

#vn = (j * df['Exit Flow Rate [sccm]'] * df['Net Ret Time (Max) [min]'] * df['Column Temperature [Kelvin]'] ) / (m * 273.15)

,1
0,7


# Partial pressure, p (Pa)

$$p=\frac{n_{i}RH_{peak}273.15}{FA_{peak}}$$

In [69]:
A_peak = df['peak_area']
n_i = df['Amount  [mMol]']
H_peak = df['peak_max_signal']
F = df['Exit Flow Rate [sccm]']

In [70]:
# Converting units to SI

vn_SI = vn * 0.001 # converts mg/L to m^3/kg
A_peak = A_peak * 6E-5 #converts uV.min to V.sec
n_i = n_i / 1000 # converts mMol to Mol
H_peak = H_peak * 1E-6 # converts uV to V
F = F * 1.6667E-8 #from sscm to m^3/s

In [71]:
# Gas constant
R = constants.R

In [72]:
p = (n_i * R * 273.15 * H_peak) / (F * A_peak) # Pa

#  Amount of probe adsorbed on sample surface, n, ($\frac{mMol}{g}$)

$$n=\frac{1}{RT}\intop_{0}^{p}V_{n}dp$$

In [73]:
# Reindexing series to start at zero
p_reindex = p.reset_index(drop=True)
vn_SI_reindex = vn_SI.reset_index(drop=True)

In [74]:
# T value used in n calc
T_av = df['Column Temperature [Kelvin]'].mean()

In [75]:
# set n as empty array in shape of as other values 
n = np.empty_like(p_reindex)

In [76]:
# Cumulative integration  of vn with respect to partial pressure p
n[0] = (p_reindex[0] * vn_SI_reindex[0]) / (R * T_av)
for i in range(1, len(p_reindex)):
    n[i] = (p_reindex[i] - p_reindex[i - 1]) * (vn_SI_reindex[i] / (R * T_av)) + n[i - 1]

KeyError: 0L

In [ ]:
# set n as series with same index as p_reindex and vn_SI_reindex
n = Series(n)

# Saturation vapour pressure calculation, $p_{sat}$ (Pa)

$$\log_{10}p_{sat}=A-\frac{B}{C+T}$$
$$p_{sat}=10^{(A-\frac{B}{C+T})}$$

In [ ]:
# Antoine coefficients for octane (probe used for surface area calculations from AntoineConstants excel file)

In [ ]:
# Coefficents A,B,C are for T = 20C
A = 6.91868
B = 1351.99
C = 209.155


p_sat = 10 ** (A- (B /(20+C))) # mmHg T= 20C
p_sat = p_sat * 133.322 # mmHg to Pa

# Relative pressure, $p_{rel}$

$$p_{rel}=\frac{p}{p_{sat}}$$

In [ ]:
p_rel = p_reindex / p_sat

# Adsorption Isotherm Plot

In [ ]:
plt.scatter(p_rel, n)
plt.title('Adsorption Isotherm')
plt.xlabel('Relative Pressure (p/p_rel)')
plt.ylabel('Amount Adsorbed (mMol/g)')
x_max = max(p_rel) + (0.1 * max(p_rel))
y_max = max(n) + (0.1 * max(n))
plt.ylim(0,y_max)
plt.xlim(0,x_max)

#  BET  equation plot and calculation

$$\frac{p}{n(p-p_{sat})}=\frac{C-1}{n_{m}C}\frac{p}{p_{sat}}+\frac{1}{n_{m}C}$$

In [ ]:
# setting Y axis for BET plot
y_BET = p_reindex / (n * (p_sat - p_reindex))

In [ ]:
# set up BET dataframe
BET_df = pd.concat([p_rel, y_BET], axis = 1)
BET_df.columns = ['p_rel', 'y_BET']

In [ ]:
# BET equation is only valid between p_rel values of 0.05 to 0.35
BET_df = BET_df[(BET_df.p_rel >= 0.05) & (BET_df.p_rel <= 0.35)]

In [ ]:
BET_df

In [ ]:
# Function does linear regression and if threshold r_sq values isnt reached it removes the first row of dataframe
# First row is removed because generally BET data is better at higher p_rel
def regress(df):
    # Linear regression of BET plot within allowed p_rel range
    slope, intercept, r_value, p_value, std_err = stats.linregress(df['p_rel'],df['y_BET'])
    
    r_sq = r_value ** 2
    
    if r_sq < 0.995:
        
        return regress(df.iloc[1:]) # re-runs function with first row removed, removes first row until threshold r_sq reached
    else:
        # Calculations used for finding error in intercept from 'https://en.wikipedia.org/wiki/Regression_analysis'
        sse = ((df['y_BET'] - ((slope * df['p_rel']) + intercept)) ** 2).sum()
        n = len(df['p_rel'])
        var = np.sqrt((sse / (n - 2)))
        x_mean = df['p_rel'].mean()
        sx2 = ((df['p_rel'] - x_mean) **2).sum()
        intercept_err = var * np.sqrt((1/n) + (x_mean ** 2) / sx2)
        
        return slope,intercept,r_value,p_value,std_err,r_sq, intercept_err

In [ ]:
slope,intercept,r_value,p_value, std_err, r_sq,intercept_err = regress(BET_df)

In [ ]:
# BET plot
plt.scatter(p_rel,y_BET)
plt.plot(p_rel, (p_rel * slope) + intercept, c='r')
plt.title('BET plot')
plt.xlabel('Relative Pressure (p/p_rel)')
plt.ylabel('p / n[p_sat-p] (g/mMol)')


$$n_{m}=\frac{1}{slope+intercept}$$

In [ ]:
# The monolayer capacity, n_m is found from slope and intercept of fitted line from BET plot

n_m = 1 / (slope + intercept) # mMol/g
err_slope_intercept = np.sqrt((std_err) ** 2 + (intercept_err) ** 2) # error in slope+intercept
n_m_err = n_m * (err_slope_intercept / (slope + intercept)) # error in n_m
n_m = n_m / 1000 # change units from mMol/g to Mol/g
n_m_err = n_m_err / 1000 # change units from mMol/g to Mol/g

#  Specific Surface Area, $\frac{m^{2}}{g}$

$$S_{BET}=n_{m}N_{a}a$$

In [ ]:
a = 6.3E-19 # m^2 This is the cross-sectional surface area of a Octane molecule
Na = constants.Avogadro

In [ ]:
ssa = n_m * Na * a
ssa_err = ssa * (n_m_err / n_m)

In [ ]:
print 'The SSA is %.2f with error %.2f and R^2 of %.4f' % (ssa,ssa_err,r_sq)